In [1]:
from AttentionModule import Conv2d_Attn

import torch
from torch import nn
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import re
import numpy as np
import time
from tqdm import tqdm

In [2]:
resnet_pretrained = models.resnet50(pretrained=True)
nn.Conv2d = Conv2d_Attn
resnet_attn = models.resnet50()
    
resnet_attn.load_state_dict(resnet_pretrained.state_dict(), strict=False)
# print(resnet_pretrained.state_dist())

In [3]:
# This block turns 'layer1.0.downsample.0.weight' to 'layer1[0].downsample[0].weight'
param_keys = list(resnet_attn.state_dict().keys())
formatted_keys = []
for k in param_keys:
    found = re.findall(r'\.[\d]{1,2}\.', k)
    if len(found):
        for f in found:
            k = k.replace(f, '[{}].'.format(f.strip('.')))
    formatted_keys.append(k)

In [4]:
# This block turn off gradient up for all params except attn_weights
def turn_off_grad_except(lst=[]):
    turning_off_keys = [k for k in formatted_keys for l in lst if l in k]
    for k in formatted_keys:
        obj = eval('resnet_attn.'+k)
        if k in turning_off_keys:
            print(k)
            obj.requires_grad = True
        else:
            obj.requires_grad = False
    

def turn_on_layer4_conv_weight():
    for k in range(3):
        for j in range(1, 4):
            eval('resnet_attn.layer4[{}].conv{}.weight'.format(k, j)).requires_gred = True

In [5]:
resnet_attn.fc = nn.Linear(resnet_attn.fc.in_features, 144)

Start training

In [6]:
batch_size = 32

In [7]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [8]:
total_imgs = len(trainset.imgs)

In [9]:
resnet_attn = resnet_attn.cuda()

In [10]:
total_attn_params = 0
for k in formatted_keys:
    obj = eval('resnet_attn.'+k)
    if 'attn_weights' in k:
        total_attn_params += np.prod(obj.shape)
print("Total number of attention parameters", total_attn_params)

Total number of attention parameters 26560


We want the attention parameters to diverge from 1, therefore we penalize element-wise square loss as $\lambda (1 \times \text{# params} - (x - 1)^2)$

But this is too big a number,
let's try: 
$- (x - 1)^2$ for now

In [11]:
_lambda = 1e-1 #set default

In [12]:
def get_params_objs(name, net='resnet_attn'):
    res = []
    for k in formatted_keys:
        obj = eval('{}.'.format(net)+k)
        if name in k:
            res.append(obj)
    return res

In [13]:
def compute_attn_loss(n_params=26560):
    attns = get_params_objs('attn_weights')
#     penality = sum([torch.pow(t - 1,2).mean() for t in attns])
    penality = (1000/n_params)*sum([torch.min(torch.pow(t-2, 2), torch.pow(t, 2)).sum() for t in attns])
    return (_lambda)*(penality)

In [14]:
len(get_params_objs('attn_weights'))

53

In [15]:
compute_attn_loss()

tensor(100.0000, device='cuda:0')

In [16]:
print_every = 20

In [26]:
def train_k_epoch(k, add_attn=True, score_epoch=False):
    cls_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet_attn.parameters()))
    
    start = time.time()
    
    for epoch in range(k):
        running_loss = 0.0
        running_attn_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

            optimizer.zero_grad()
            outputs = resnet_attn(inputs)
            clf_loss = cls_criterion(outputs, labels)
                        
            attn_loss = compute_attn_loss()
            
            if add_attn:
                loss = clf_loss + attn_loss
            else:
                loss = clf_loss

            loss.backward()
            optimizer.step()

            running_loss += clf_loss.data[0]
            running_attn_loss += attn_loss.data[0]

            if i % print_every == 0:
                print('[%5d] iter, [%2f] epoch, classifer loss: %.3f, attn_loss: %.5f' %
                      (i + 1, i*batch_size/total_imgs, running_loss/print_every, running_attn_loss/print_every))
                running_loss = 0.0
                running_attn_loss = 0.0
        if score_epoch:
            dic = score(batch_size=32)
            e_time = time.time() - start
            dic['time_elapsed'] = '{:.0f}m {:.0f}s'.format(e_time // 60, e_time%60)
            print(dic)

In [24]:

def score(net=resnet_attn, batch_size=batch_size):
    trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    valset = torchvision.datasets.ImageFolder(root='./data/val', transform=transform)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    train_correct = 0
    val_correct_top1 = 0
    val_correct_top3 = 0
    val_correct_top5 = 0
    
    for inp, label in tqdm(iter(valloader)):
        _, idx = net(Variable(inp).cuda()).topk(1)
        val_correct_top1 += int(sum([label[i] in idx.cpu().data[i] for i in range(len(idx))]))
        
        _, idx = net(Variable(inp).cuda()).topk(3)
        val_correct_top3 += int(sum([label[i] in idx.cpu().data[i] for i in range(len(idx))]))
        
        _, idx = net(Variable(inp).cuda()).topk(5)
        val_correct_top5 += int(sum([label[i] in idx.cpu().data[i] for i in range(len(idx))]))
        
    for inp, label in tqdm(iter(trainloader)):
        _, idx = net(Variable(inp).cuda()).topk(1)
        train_correct += int(sum([label[i] in idx.cpu().data[i] for i in range(len(idx))]))
    
    return {
        'train_accu': train_correct/len(trainset),
        'val_accu_top1': val_correct_top1/len(valset),
        'val_accu_top3': val_correct_top3/len(valset),
        'val_accu_top5': val_correct_top5/len(valset)
    }

Train a fresh fc layer. 
`turn_off_grad_except([])` turns off grads for all weights but the fc layer

In [ ]:
turn_off_grad_except(['fc'])
turn_on_layer4_conv_weight()
resnet_attn.eval()
resnet_attn.layer4.train()
train_k_epoch(50,score_epoch=True, add_attn=False)

fc.weight
fc.bias


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[    1] iter, [0.000000] epoch, classifer loss: 0.147, attn_loss: 5.00000
[   21] iter, [0.078422] epoch, classifer loss: 3.365, attn_loss: 100.00001
[   41] iter, [0.156844] epoch, classifer loss: 2.824, attn_loss: 100.00001
[   61] iter, [0.235265] epoch, classifer loss: 2.821, attn_loss: 100.00001
[   81] iter, [0.313687] epoch, classifer loss: 2.784, attn_loss: 100.00001
[  101] iter, [0.392109] epoch, classifer loss: 2.722, attn_loss: 100.00001
[  121] iter, [0.470531] epoch, classifer loss: 2.623, attn_loss: 100.00001
[  141] iter, [0.548952] epoch, classifer loss: 2.691, attn_loss: 100.00001
[  161] iter, [0.627374] epoch, classifer loss: 2.727, attn_loss: 100.00001
[  181] iter, [0.705796] epoch, classifer loss: 2.566, attn_loss: 100.00001
[  201] iter, [0.784218] epoch, classifer loss: 2.613, attn_loss: 100.00001
[  221] iter, [0.862639] epoch, classifer loss: 2.621, attn_loss: 100.00001
[  241] iter, [0.941061] epoch, classifer loss: 2.507, attn_loss: 100.00001


100%|██████████| 256/256 [01:07<00:00,  3.78it/s]

{'val_accu_top5': 0.6180159635119726, 'train_accu': 0.43389290528121555, 'time_elapsed': '2m 59s', 'val_accu_top3': 0.5273660205245154, 'val_accu_top1': 0.3403648802736602}


...

{'val_accu_top5': 0.645381984036488, 'train_accu': 0.8273495895110893, 'time_elapsed': '125m 21s', 'val_accu_top3': 0.5672748004561003, 'val_accu_top1': 0.35290763968072975}


[    1] iter, [0.000000] epoch, classifer loss: 0.027, attn_loss: 5.00000
[   21] iter, [0.078422] epoch, classifer loss: 0.830, attn_loss: 100.00001
[   41] iter, [0.156844] epoch, classifer loss: 0.703, attn_loss: 100.00001
[   61] iter, [0.235265] epoch, classifer loss: 0.539, attn_loss: 100.00001
[   81] iter, [0.313687] epoch, classifer loss: 0.603, attn_loss: 100.00001
[  101] iter, [0.392109] epoch, classifer loss: 0.676, attn_loss: 100.00001
[  121] iter, [0.470531] epoch, classifer loss: 0.600, attn_loss: 100.00001
[  141] iter, [0.548952] epoch, classifer loss: 0.575, attn_loss: 100.00001
[  161] iter, [0.627374] epoch, classifer loss: 0.627, attn_loss: 100.00001
[  181] iter, [0.705796] epoch, classifer loss: 0.595, attn_loss: 100.00001
[  201] iter, [0.784218] epoch, classifer loss: 0.573, attn_loss: 100.00001
[  221] iter, [0.862639] epoch, classifer loss: 0.598, attn_loss: 100.00001
[  241] iter, [0.941061] epoch, classifer loss: 0.660, attn_loss: 100.00001


100%|██████████| 256/256 [01:07<00:00,  3.77it/s]

{'val_accu_top5': 0.6619156214367161, 'train_accu': 0.847812768043132, 'time_elapsed': '128m 20s', 'val_accu_top3': 0.5689851767388826, 'val_accu_top1': 0.34207525655644244}


[    1] iter, [0.000000] epoch, classifer loss: 0.023, attn_loss: 5.00000
[   21] iter, [0.078422] epoch, classifer loss: 0.756, attn_loss: 100.00001
[   41] iter, [0.156844] epoch, classifer loss: 0.559, attn_loss: 100.00001
[   61] iter, [0.235265] epoch, classifer loss: 0.640, attn_loss: 100.00001
[   81] iter, [0.313687] epoch, classifer loss: 0.537, attn_loss: 100.00001
[  101] iter, [0.392109] epoch, classifer loss: 0.615, attn_loss: 100.00001
[  121] iter, [0.470531] epoch, classifer loss: 0.558, attn_loss: 100.00001


100%|██████████| 55/55 [00:14<00:00,  3.79it/s]


{'train_accu': 0.0, 'val_accu': 0.5872291904218928}

In [ ]:
compute_attn_loss()

In [ ]:
_lambda = 0.01

In [ ]:
turn_off_grad_except(['attn_weights','fc', 'layer4'])
resnet_attn.eval()
train_k_epoch(5,score_epoch=True)

In [28]:
3

3